# Creating a deepstream pipeline

In [1]:
import sys
sys.path.append('../')
import gi
import configparser
gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst
from ctypes import *
import time
import sys
import math
import platform
from common.is_aarch_64 import is_aarch64
from common.bus_call import bus_call
from common.FPS import PERF_DATA

import cv2
import numpy as np

In [2]:
import pyds

In [3]:
import gi

perf_data = None

MAX_DISPLAY_LEN=64
pgie_classes_str= ["Vehicle", "Person", "Bike", "Roadsign"]

MUXER_OUTPUT_WIDTH=1920
MUXER_OUTPUT_HEIGHT=1080
MUXER_BATCH_TIMEOUT_USEC=4000000

TILED_OUTPUT_WIDTH=1280
TILED_OUTPUT_HEIGHT=720

GST_CAPS_FEATURES_NVMM="memory:NVMM"

OSD_PROCESS_MODE= 0
OSD_DISPLAY_TEXT= 1

args = ["testing.py", "/opt/nvidia/deepstream/deepstream-6.1/samples/streams/sample_qHD.h264", "/home/mher/projects/counter/sample_pyout.h264"]

In [4]:
from common.is_aarch_64 import is_aarch64
from common.bus_call import bus_call
from common.FPS import PERF_DATA

PGIE_CLASS_ID_VEHICLE = 2
PGIE_CLASS_ID_PERSON = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_ROADSIGN = 3

TILED_OUTPUT_WIDTH = 1920
TILED_OUTPUT_HEIGHT = 1080

In [5]:
def osd_sink_pad_buffer_probe(pad,info,u_data):
    frame_number=0
    #Intiallizing object counter with 0.
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE:0,
        PGIE_CLASS_ID_PERSON:0,
        PGIE_CLASS_ID_BICYCLE:0,
        PGIE_CLASS_ID_ROADSIGN:0
    }
    num_rects=0

    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            # The casting is done by pyds.glist_get_nvds_frame_meta()
            # The casting also keeps ownership of the underlying memory
            # in the C code, so the Python garbage collector will leave
            # it alone.
            #frame_meta = pyds.glist_get_nvds_frame_meta(l_frame.data)
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                #obj_meta=pyds.glist_get_nvds_object_meta(l_obj.data)
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            obj_counter[obj_meta.class_id] += 1
#             print(f"Detected {pgie_classes_str[obj_meta.class_id]}")
            obj_meta.rect_params.border_color.set(0.0, 0.0, 1.0, 0.0)
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break

        # Acquiring a display meta object. The memory ownership remains in
        # the C code so downstream plugins can still access it. Otherwise
        # the garbage collector will claim it when this probe function exits.
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        # Note that the pyds module allocates a buffer for the string, and the
        # memory will not be claimed by the garbage collector.
        # Reading the display_text field here will return the C address of the
        # allocated string. Use pyds.get_string() to get the string content.
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Vehicle_count={} Person_count={}".format(frame_number, num_rects, obj_counter[PGIE_CLASS_ID_VEHICLE], obj_counter[PGIE_CLASS_ID_PERSON])

        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12

        # Font , font-color and font-size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        # set(red, green, blue, alpha); set to White
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)

        # Text background color
        py_nvosd_text_params.set_bg_clr = 1
        # set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Using pyds.get_string() to get display_text as string
        print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        try:
            l_frame=l_frame.next
        except StopIteration:
            break

    return Gst.PadProbeReturn.OK

In [6]:
# Standard GStreamer initialization
Gst.init(None)

# Create gstreamer elements
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n ")
pipeline = Gst.Pipeline()
if not pipeline:
        print(" Unable to create Pipeline \n")

Creating Pipeline 
 


In [7]:
# Source element for reading from the file
print("Creating Source \n ")
source = Gst.ElementFactory.make("filesrc", "file-source")
if not source:
    print(" Unable to create Source \n")

Creating Source 
 


In [8]:
# Since the data format in the input file is elementary h264 stream,
# we need a h264parser
print("Creating H264Parser \n")
h264parser = Gst.ElementFactory.make("h264parse", "h264-parser")
if not h264parser:
    print(" Unable to create h264 parser \n")

Creating H264Parser 



In [9]:
# Use nvdec_h264 for hardware accelerated decode on GPU
print("Creating Decoder \n")
decoder = Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")
if not decoder:
    print(" Unable to create Nvv4l2 Decoder \n")

Creating Decoder 



In [10]:
# Create nvstreammux instance to form batches from one or more sources.
print("Creating Muxer \n")
streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
if not streammux:
    print(" Unable to create NvStreamMux \n")

Creating Muxer 



In [11]:
# Use nvinfer to run inferencing on decoder's output,
# behaviour of inferencing is set through config file
print("Creating pgie")
pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
if not pgie:
    print(" Unable to create pgie \n")

Creating pgie


In [12]:
# Use convertor to convert from NV12 to RGBA as required by nvosd
print("Creating nvvidconv")
nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
if not nvvidconv:
    print(" Unable to create nvvidconv \n")

Creating nvvidconv


In [13]:
# Create OSD to draw on the converted RGBA buffer
print("Creating OSD")
nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
if not nvosd:
        print(" Unable to create nvosd \n")

Creating OSD


In [14]:
# Use convertor to convert from something to something
print("Creating encoder")
enc = Gst.ElementFactory.make("nvv4l2h264enc", "nvv4l2h264enc")
if not nvvidconv:
    print(" Unable to create nvvidconv \n")
    
# # Use convertor to convert from something to something
# print("Creating encoder")
# nvvidconv2 = Gst.ElementFactory.make("nvvideoconvert", "nvv4l2h264enc")
# if not nvvidconv:
#     print(" Unable to create nvvidconv \n")

print("Creating File sink \n")
sink = Gst.ElementFactory.make("filesink", "nvvideo-renderer")
if not sink:
    print(" Unable to create file sink \n")

# print("Creating EGL sink \n")
# sink = Gst.ElementFactory.make("nveglglessink", "nvvideo-renderer")
# if not sink:
#     print(" Unable to create egl sink \n")

# print("Creating fake sink \n")
# sink = Gst.ElementFactory.make("fakesink", "nvvideo-renderer")
# if not sink:
#     print(" Unable to create fake sink \n")

Creating encoder
Creating File sink 



In [15]:
print("Playing file %s " %args[1])
source.set_property('location', args[1])
sink.set_property("location", args[2])
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
streammux.set_property('batched-push-timeout', 4000000)
pgie.set_property('config-file-path', "/home/mher/projects/counter/deepstream-analytics/dsnvanalytics_pgie_config.txt")


print("Adding elements to Pipeline \n")
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(enc)
pipeline.add(sink)
# if is_aarch64():
#     pipeline.add(transform)

Playing file /opt/nvidia/deepstream/deepstream-6.1/samples/streams/sample_qHD.h264 
Adding elements to Pipeline 



True

In [16]:
# we link the elements together
# file-source -> h264-parser -> nvh264-decoder ->
# nvinfer -> nvvidconv -> nvosd -> video-renderer
print("Linking elements in the Pipeline \n")

source.link(h264parser)
h264parser.link(decoder)

sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    print(" Unable to get the sink pad of streammux \n")
    
srcpad = decoder.get_static_pad("src")
if not srcpad:
    print(" Unable to get source pad of decoder \n")
    
srcpad.link(sinkpad)
streammux.link(pgie)
pgie.link(nvvidconv)

# nvvidconv.link(sink)
# nvosd.link(sink)
nvvidconv.link(nvosd)
nvosd.link(enc)
enc.link(sink)

Linking elements in the Pipeline 



True

In [17]:
# create an event loop and feed gstreamer bus mesages to it
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

1

In [18]:
# Lets add probe to get informed of the meta data generated, we add probe to
# the sink pad of the osd element, since by that time, the buffer would have
# had got all the metadata.
osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    print(" Unable to get sink pad of nvosd \n")

osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)

1

In [19]:
# start play back and listen to events
print("Starting pipeline \n")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except Exception as e:
    print(e)
# cleanup
pipeline.set_state(Gst.State.NULL)

Starting pipeline 

Frame Number=0 Number of Objects=6 Vehicle_count=1 Person_count=4
Frame Number=1 Number of Objects=6 Vehicle_count=1 Person_count=4


Error: gst-stream-error-quark: Internal data stream error. (1): gstnvinfer.cpp(2299): gst_nvinfer_output_loop (): /GstPipeline:pipeline0/GstNvInfer:primary-inference:
streaming stopped, reason not-linked (-1)


<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>

In [20]:
# from os.path import exists

# file_exists = exists(path_to_file)

dir(pipeline)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__gdoc__',
 '__ge__',
 '__getattribute__',
 '__gpointer__',
 '__grefcount__',
 '__gsignals__',
 '__gt__',
 '__gtype__',
 '__hash__',
 '__info__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_force_floating',
 '_gst_reserved',
 '_ref',
 '_ref_sink',
 '_unref',
 '_unsupported_data_method',
 '_unsupported_method',
 'abort_state',
 'add',
 'add_control_binding',
 'add_metadata',
 'add_pad',
 'add_pad_template',
 'add_property_deep_notify_watch',
 'add_property_notify_watch',
 'add_static_metadata',
 'add_static_pad_template',
 'add_static_pad_template_with_gtype',
 'auto_clock',
 'base_time',
 'bin',
 'bind_property',
 'bind_property_full',
 'bus',
 'call_async',
 'chain',
 'change_state',
 

In [21]:
for i in range(pipeline.get_children_count()):
    print(pipeline.get_child_by_index(i))

<__gi__.GstFileSink object at 0x7f6649102940 (GstFileSink at 0x36e49f0)>
<__gi__.nvv4l2h264enc object at 0x7f66491023c0 (nvv4l2h264enc at 0x36de470)>
<__gi__.GstNvDsOsd object at 0x7f66490fddc0 (GstNvDsOsd at 0x362b8d0)>
<__gi__.Gstnvvideoconvert object at 0x7f66490fd880 (Gstnvvideoconvert at 0x35e91c0)>
<__gi__.GstNvInfer object at 0x7f66490fd2c0 (GstNvInfer at 0x35e2ba0)>
<__gi__.GstNvStreamMux object at 0x7f66490f3bc0 (GstNvStreamMux at 0x2a54160)>
<__gi__.nvv4l2decoder object at 0x7f66490f3600 (nvv4l2decoder at 0x2945140)>
<__gi__.GstH264Parse object at 0x7f6649104fc0 (GstH264Parse at 0x2939f90)>
<__gi__.GstFileSrc object at 0x7f6649104a40 (GstFileSrc at 0x2932170)>
